In [212]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [213]:
df = pd.read_csv(r'C:\Users\Hp\OneDrive\Desktop\MRS\Song_Recommendation\Datasets\spotify_millsongdata.csv')

In [214]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [215]:
df.drop('link', inplace=True, axis=1)

In [216]:
df.head()

,artist,song,text
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...


In [217]:
df.shape

(57650, 3)

In [218]:
df = df.sample(10000)

In [219]:
df.isnull().sum()

artist    0
song      0
text      0
dtype: int64

In [220]:
df.loc[:, 'text'] = df['text'].apply(lambda x: x.lower().replace('\r','').replace('\n',''))

In [221]:
# Initialize PorterStemmer
porter = PorterStemmer()

# Get the set of English stopwords
stop_words = set(stopwords.words('english'))

# Define a function for preprocessing a sentence
def preprocess_sentence(sentence):
    # Tokenize the input sentence into words
    words = word_tokenize(sentence)
    
    # Stem each word using PorterStemmer and filter out stopwords
    stemmed_words = [porter.stem(word) for word in words if word.lower() not in stop_words]
    
    # Join the stemmed words back into a single string
    return ' '.join(stemmed_words)

In [222]:
df.loc[:, 'text'] = df['text'].apply(preprocess_sentence)

In [223]:
df.loc[:, 'text'] = df['text'].apply(lambda x: x.replace("'", ''))

In [224]:
df.reset_index(drop=True, inplace=True)

In [225]:
df.head()

,artist,song,text
0,Used,The Bird And The Worm,wear heart safeti pin backpack backpack know s...
1,George Strait,Hollywood Squares,"ve alway said `` easi , m payin price . weak..."
2,Incubus,Get Your Dreidel On,"yo , s hanukkah time right [ ? ] dreidlein bo..."
3,Nina Simone,"Papa, Can You Hear Me?","papa , hear ? papa , see ? papa , find night ?..."
4,U2,I Will Follow,"follow outsid said said need look blind , coul..."


In [226]:
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])

In [227]:
similarity_matrix = cosine_similarity(tfidf_matrix)

MemoryError: Unable to allocate 727. MiB for an array with shape (95344150,) and data type float64

In [ ]:
def recommendation(song_df):
    # Check if the song exists in the DataFrame
    if song_df not in df['song'].values:
        print(f"The song '{song_df}' does not exist in the DataFrame.")
        return []

    # Find the index of the song in the DataFrame
    idx = df[df['song'] == song_df].index[0]
    
    # Sort the similarity scores in descending order
    distances = sorted(list(enumerate(similarity_matrix[idx])), reverse=True, key=lambda x: x[1])
    
    # Get the indices of the top 5 most similar songs
    recommended_indices = [m_id[0] for m_id in distances[1:6]]
    
    # Get the titles of the recommended songs
    recommended_songs = df.iloc[recommended_indices]['song'].tolist()
    
    return recommended_songs


In [228]:
recommended_songs = recommendation("Dark")
print("Recommended songs:")
for song in recommended_songs:
    print(song)

Recommended songs:
Now That You're Dead
Make The World Move
One More Chance
I Came To Believe
Broken Man
